In [ ]:
%pip install yfinance prophet plotly

import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from prophet import Prophet
from prophet.plot import plot_plotly

# Fetch historical data
amd = yf.Ticker("AMD")
data = amd.history(period="5y")

# Calculate technical indicators
data['SMA50'] = data['Close'].rolling(window=50).mean()
data['SMA200'] = data['Close'].rolling(window=200).mean()
data['RSI'] = 100 - (100 / (1 + data['Close'].diff(1).clip(lower=0).rolling(14).mean() / 
                            data['Close'].diff(1).clip(upper=0).abs().rolling(14).mean()))

# Create interactive chart
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.03, subplot_titles=('AMD Stock Price', 'Volume'), 
                    row_width=[0.7, 0.3])


fig.add_trace(go.Scatter(x=data.index, y=data['SMA50'], name='SMA 50', line=dict(color='orange', width=1)),
              row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['SMA200'], name='SMA 200', line=dict(color='green', width=1)),
              row=1, col=1)

fig.add_trace(go.Bar(x=data.index, y=data['Volume'], name='Volume'), row=2, col=1)

fig.update_layout(title='AMD Stock Analysis', xaxis_rangeslider_visible=False)
fig.show()

# Prepare data for Prophet
df = data.reset_index()[['Date', 'Close']]
df.columns = ['ds', 'y']
df['ds'] = df['ds'].dt.tz_localize(None)

# Create and fit the model
model = Prophet()
model.fit(df)

# Create future dates for forecasting
future_dates = model.make_future_dataframe(periods=365*5)

# Make predictions
forecast = model.predict(future_dates)

# Plot the forecast
fig = plot_plotly(model, forecast)
fig.update_layout(title='AMD Stock Price Forecast')
fig.show()

# Generate executive summary
last_price = data['Close'].iloc[-1]
forecast_1y = forecast[forecast['ds'] == forecast['ds'].max() - pd.Timedelta(days=365*4)]['yhat'].values[0]
forecast_5y = forecast[forecast['ds'] == forecast['ds'].max()]['yhat'].values[0]

print("Executive Summary:")
print(f"Current AMD stock price: ${last_price:.2f}")
print(f"Forecasted price in 1 year: ${forecast_1y:.2f}")
print(f"Forecasted price in 5 years: ${forecast_5y:.2f}")